In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA Minimization

In this notebook, we verify our minimization algorithm using a concrete example.
We manually construct a DFA, visualize it, run the minimization, and visualize the result.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from "recursive-set";

import {
    DFA,
    TransRelDet,
    DFAState,
    State,
    TransitionKey,
    key,
} from "./01-NFA-2-DFA";
import { MinDFA, minimize } from "./07-Minimize";
import { dfa2string, dfa2dot } from "./FSM-2-Dot";

const viz = await instance();

### Defining the DFA

We define the DFA $A = (Q, \Sigma, \delta, q_0, F)$ manually.

**Implementation Details:**
To ensure correct behavior with `RecursiveSet`, we pre-create the singleton sets representing our states ($\{0\}, \{1\}, \dots$). This allows us to cache them and ensures referential identity where useful, though v7 relies on structural equality.

**Transitions:**
* $(0, a) \to 1, \quad (0, b) \to 6$
* ... (see code for full definition)

In [ ]:
// 1. Pre-create ALL state instances (Singletons {0}..{8})
const states: DFAState[] = Array.from({ length: 9 }, (_, i) =>
    RecursiveSet.fromSortedUnsafe([i]),
);

// Helper: Access the cached instance by index
const S = (i: number) => states[i];

const Sigma = new RecursiveSet<string>("a", "b");
const delta = new Map<TransitionKey, DFAState>();

const addTrans = (from: number, char: string, to: number) => {
    delta.set(key(S(from), char), S(to));
};

// Define transitions
addTrans(0, "a", 1);
addTrans(0, "b", 6);
addTrans(1, "a", 2);
addTrans(1, "b", 7);
addTrans(2, "a", 3);
addTrans(2, "b", 6);
addTrans(3, "a", 0);
addTrans(3, "b", 7);
addTrans(4, "a", 5);
addTrans(4, "b", 2);
addTrans(5, "a", 6);
addTrans(5, "b", 8);
addTrans(6, "a", 7);
addTrans(6, "b", 2);
addTrans(7, "a", 0);
addTrans(7, "b", 3);
addTrans(8, "a", 4);
addTrans(8, "b", 8);

const Q = RecursiveSet.fromArray(states);
const A_states = RecursiveSet.fromArray([S(2), S(3), S(6), S(7)]);
const q0 = S(0);

const A: DFA = {
    Q: Q,
    Sigma: Sigma,
    delta: delta,
    q0: q0,
    A: A_states,
};

### Visualize Original DFA

We generate both the text representation (to see the state encoding) and the Graphviz diagram.

**Note:** The state names (S0, S1...) in the graph are generated automatically. Check the "state encoding" in the text output to map them back to our sets (e.g., see which S-name corresponds to `{0}`).

In [ ]:
console.log(dfa2string(A));

const { dot: dot1 } = dfa2dot(A);
display.html(viz.renderString(dot1, { format: "svg" }));

### Minimize and Visualize

We apply the `minimize` function to compute the **Quotient Automaton**.

**Mathematical Structure:**
The result is a `MinDFA`, where each state is an **Equivalence Class** (a set of original states).
* Example: If states $\{2\}$ and $\{6\}$ are equivalent, the new state is the set `{{2}, {6}}`.

**Visualization Note:**
Although `dfa2string` and `dfa2dot` are typed for standard `DFA`s (sets of primitives), they work perfectly with `MinDFA` because `RecursiveSet` handles nested structures gracefully. We simply cast the result to `DFA` to satisfy the TypeScript compiler. The output below clearly shows the nested sets structure.

In [ ]:
// We cast immediately to DFA to allow easy usage with dfa2dot/dfa2string
const F = minimize(A) as unknown as DFA;

### Visualize Minimized DFA

Now we can use our standard tools without extra casting.

The output shows the merged states (e.g., `{{3}, {7}}`).

In [ ]:
console.log("Minimized DFA Structure:");
console.log(dfa2string(F));

const { dot: dot2 } = dfa2dot(F);
display.html(viz.renderString(dot2, { format: "svg" }));